## Fine-tuning mô hình SER trên bộ ViSEC

Notebook này sử dụng dữ liệu đã được chuẩn bị ở `prepare_data_SER_VISEC.ipynb` (lưu tại `./data/ser_visec`) để fine-tune mô hình `superb/hubert-large-superb-er` cho bài toán nhận diện cảm xúc trong giọng nói tiếng Việt.

Các bước chính:

1. Tải lại các split `train`, `val`, `test` từ thư mục `./data/ser_visec` bằng `load_from_disk`.
2. Load mô hình và feature extractor `superb/hubert-large-superb-er`.
3. Cấu hình lại đầu ra của mô hình với 3 lớp cảm xúc: `positive`, `negative`, `neutral`.
4. Tiền xử lý:
   - Chuyển audio (`array`, `sampling_rate`) thành `input_values`, `attention_mask`.
   - Mã hóa nhãn cảm xúc thành các chỉ số 0, 1, 2.
5. Thiết lập `DataCollatorWithPadding` cho batch.
6. Thiết lập `TrainingArguments` và `Trainer` để huấn luyện.
7. Huấn luyện trên tập train, đánh giá trên tập validation (metrics: Accuracy, F1-score).
8. Đánh giá cuối trên tập test, in thêm Confusion Matrix / Classification Report nếu cần.
9. Lưu mô hình đã fine-tune vào `./models/hubert-ser-finetuned-visec-final`.


In [ ]:
# Cài đặt thư viện cần thiết (chạy một lần trong môi trường mới)
# Nếu môi trường đã có các thư viện này, có thể bỏ qua cell này
%pip install -q transformers datasets scikit-learn

from datasets import load_from_disk
from transformers import (
    AutoModelForAudioClassification,
    AutoFeatureExtractor,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from pathlib import Path
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Thiết bị sử dụng:", device)

# 1. Tải dữ liệu SER đã chuẩn bị
print("\n[1/6] Đang tải dữ liệu SER từ ./data/ser_visec ...")
DATA_DIR = Path("./data/ser_visec")
train_dataset = load_from_disk(str(DATA_DIR / "train"))
val_dataset = load_from_disk(str(DATA_DIR / "val"))
test_dataset = load_from_disk(str(DATA_DIR / "test"))

print("Số mẫu:")
print("  Train:", len(train_dataset))
print("  Val:  ", len(val_dataset))
print("  Test: ", len(test_dataset))

# 2. Load mô hình và feature extractor HuBERT
print("\n[2/6] Đang tải mô hình HuBERT cho SER...")
SER_MODEL_ID = "superb/hubert-large-superb-er"

feature_extractor = AutoFeatureExtractor.from_pretrained(SER_MODEL_ID)

# Cấu hình lại đầu ra với 3 lớp
label2id = {"positive": 0, "negative": 1, "neutral": 2}
id2label = {v: k for k, v in label2id.items()}

model = AutoModelForAudioClassification.from_pretrained(
    SER_MODEL_ID,
    num_labels=3,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

model.to(device)

print("Đã tải mô hình:", SER_MODEL_ID)
print("Số lớp cảm xúc:", model.config.num_labels)

# 3. Tiền xử lý dữ liệu: audio → input_values, labels
print("\n[3/6] Đang tiền xử lý dữ liệu SER...")

def prepare_ser_batch(batch):
    """Tiền xử lý 1 sample cho SER HuBERT."""
    audio = batch["audio"]

    inputs = feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_tensors="pt",
        padding=True,
    )

    batch["input_values"] = inputs["input_values"][0]
    # Một số feature_extractor không trả về attention_mask; kiểm tra trước
    if "attention_mask" in inputs:
        batch["attention_mask"] = inputs["attention_mask"][0]

    # Mã hóa nhãn cảm xúc
    batch["labels"] = label2id[str(batch["emotion"]).lower().strip()]
    return batch

train_proc = train_dataset.map(
    prepare_ser_batch,
    remove_columns=train_dataset.column_names,
    num_proc=4,
)
val_proc = val_dataset.map(
    prepare_ser_batch,
    remove_columns=val_dataset.column_names,
    num_proc=4,
)
test_proc = test_dataset.map(
    prepare_ser_batch,
    remove_columns=test_dataset.column_names,
    num_proc=4,
)

print("Ví dụ 1 sample sau tiền xử lý:")
print("  input_values shape:", train_proc[0]["input_values"].shape)
print("  label:", train_proc[0]["labels"])

# 4. Data collator
print("\n[4/6] Thiết lập DataCollatorWithPadding...")

data_collator = DataCollatorWithPadding(
    tokenizer=feature_extractor,
    padding=True,
)

# 5. Hàm metrics: Accuracy, F1-score (weighted)
print("\n[5/6] Định nghĩa hàm compute_metrics (Accuracy, F1-score)...")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")

    return {"accuracy": acc, "f1": f1}

# 6. Thiết lập TrainingArguments và Trainer
print("\n[6/6] Thiết lập TrainingArguments và Trainer...")

OUTPUT_DIR = Path("./models/hubert-ser-finetuned-visec")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=5,
    fp16=torch.cuda.is_available(),
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_proc,
    eval_dataset=val_proc,
    tokenizer=feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Thiết lập Trainer xong. Sẵn sàng huấn luyện.")
print("\nGọi trainer.train() trong cell tiếp theo để bắt đầu fine-tuning.")


In [ ]:
# Bắt đầu fine-tuning SER
print("Bắt đầu fine-tuning mô hình SER trên bộ ViSEC...")
train_result = trainer.train()

print("\nKết quả huấn luyện (tóm tắt):")
print(train_result)

# Đánh giá trên tập validation cuối cùng
print("\nĐánh giá trên tập validation:")
val_metrics = trainer.evaluate()
print(val_metrics)

# Đánh giá chi tiết trên tập test
print("\nĐánh giá chi tiết trên tập test...")

test_logits = trainer.predict(test_proc)
logits = test_logits.predictions
labels = test_logits.label_ids
preds = np.argmax(logits, axis=-1)

print("Metrics trên test set:")
print("  Accuracy:", accuracy_score(labels, preds))
print("  F1 (weighted):", f1_score(labels, preds, average="weighted"))

print("\nConfusion Matrix:")
print(confusion_matrix(labels, preds))

print("\nClassification Report:")
print(classification_report(labels, preds, target_names=[id2label[i] for i in range(3)]))

# Lưu mô hình fine-tuned
FINAL_DIR = Path("./models/hubert-ser-finetuned-visec-final")
FINAL_DIR.mkdir(parents=True, exist_ok=True)

print("\nĐang lưu mô hình đã fine-tune vào:", FINAL_DIR.resolve())
trainer.save_model(str(FINAL_DIR))
feature_extractor.save_pretrained(str(FINAL_DIR))

print("Hoàn thành fine-tuning SER và lưu mô hình.")
